In [3]:
# =========== PART 1: COMPREHENSIVE EDA ===========
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load and basic info - FIXED WITH YOUR FILE PATH
df = pd.read_csv('data/regression_data.csv')
print(f"Dataset: {df.shape[0]} rows, {df.shape[1]} columns")

# Check what columns we have
print(f"\nColumns in dataset:")
print(df.columns.tolist())

# 2. Check missing values
print("\n" + "="*60)
print("MISSING VALUES ANALYSIS")
print("="*60)
missing_counts = df.isnull().sum()
missing_percentage = (missing_counts / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing %': missing_percentage
}).sort_values('Missing Count', ascending=False)

print(missing_df[missing_df['Missing Count'] > 0])

if missing_counts.sum() == 0:
    print("✓ No missing values found!")

# 3. Statistical summary
print("\n" + "="*60)
print("BASIC STATISTICAL SUMMARY")
print("="*60)

# Identify numerical and categorical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

print(f"Numerical columns ({len(numerical_cols)}): {numerical_cols}")
print(f"Categorical columns ({len(categorical_cols)}): {categorical_cols}")

# Basic stats for key numerical variables (including price if it exists)
if 'price' in df.columns:
    key_vars = ['price', 'sqft_living', 'grade', 'bathrooms', 'bedrooms', 
                'sqft_lot', 'condition', 'waterfront', 'sqft_above', 
                'sqft_living15', 'sqft_lot15']
else:
    # Find similar column names
    price_col = None
    for col in df.columns:
        if 'price' in col.lower():
            price_col = col
            break
    
    if price_col:
        key_vars = [price_col, 'sqft_living', 'grade', 'bathrooms', 'bedrooms']
    else:
        # Use first 10 numerical columns
        key_vars = numerical_cols[:10]

print(f"\nBasic statistics for key variables:")
print(df[key_vars].describe().round(2))

# 4. Target variable analysis (assuming 'price' is the target)
print("\n" + "="*60)
print("TARGET VARIABLE ANALYSIS (PRICE)")
print("="*60)

# Find price column
price_col = 'price' if 'price' in df.columns else None
if not price_col:
    for col in df.columns:
        if 'price' in col.lower():
            price_col = col
            break

if price_col:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Original distribution
    axes[0].hist(df[price_col], bins=50, edgecolor='black', alpha=0.7)
    axes[0].set_title(f'{price_col} Distribution')
    axes[0].set_xlabel(price_col)
    axes[0].set_ylabel('Frequency')
    axes[0].axvline(df[price_col].mean(), color='red', linestyle='--', label=f'Mean: ${df[price_col].mean():,.0f}')
    axes[0].axvline(df[price_col].median(), color='green', linestyle='--', label=f'Median: ${df[price_col].median():,.0f}')
    axes[0].legend()
    
    # Boxplot
    axes[1].boxplot(df[price_col])
    axes[1].set_title(f'{price_col} Boxplot')
    axes[1].set_ylabel(price_col)
    
    # Log transformation (if no negative/zero prices)
    if (df[price_col] > 0).all():
        axes[2].hist(np.log1p(df[price_col]), bins=50, edgecolor='black', alpha=0.7)
        axes[2].set_title(f'Log-Transformed {price_col}')
        axes[2].set_xlabel(f'log({price_col})')
        axes[2].set_ylabel('Frequency')
    else:
        # If negative values exist, show cumulative distribution
        axes[2].hist(df[price_col], bins=50, cumulative=True, edgecolor='black', alpha=0.7)
        axes[2].set_title(f'Cumulative {price_col} Distribution')
        axes[2].set_xlabel(price_col)
        axes[2].set_ylabel('Cumulative Frequency')
    
    plt.tight_layout()
    plt.show()
    
    # Price statistics
    print(f"\n{price_col} Statistics:")
    print(f"  Mean: ${df[price_col].mean():,.2f}")
    print(f"  Median: ${df[price_col].median():,.2f}")
    print(f"  Std Dev: ${df[price_col].std():,.2f}")
    print(f"  Min: ${df[price_col].min():,.2f}")
    print(f"  Max: ${df[price_col].max():,.2f}")
    print(f"  Skewness: {df[price_col].skew():.2f}")
    print(f"  Kurtosis: {df[price_col].kurtosis():.2f}")
else:
    print("Warning: Could not find price column in dataset")
    print(f"Available columns: {df.columns.tolist()}")

# 5. Correlation analysis
print("\n" + "="*60)
print("CORRELATION ANALYSIS")
print("="*60)

if price_col and len(numerical_cols) > 1:
    # Calculate correlation
    corr_matrix = df[numerical_cols].corr()
    
    # Get correlation with price
    if price_col in corr_matrix.columns:
        price_corr = corr_matrix[price_col].sort_values(ascending=False)
        print("\nTop 10 correlations with price:")
        print(price_corr.head(10))
        
        # Visualize correlation heatmap
        plt.figure(figsize=(12, 8))
        top_corr_features = price_corr.index[:10].tolist()
        top_corr_matrix = df[top_corr_features].corr()
        
        sns.heatmap(top_corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                    center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
        plt.title('Top 10 Features Correlation with Price', fontsize=14)
        plt.tight_layout()
        plt.show()
    else:
        print(f"Price column '{price_col}' not found in numerical columns")
else:
    print("Not enough numerical columns for correlation analysis")

# 6. High-value properties analysis (≥ $650K)
print("\n" + "="*60)
print("HIGH-VALUE PROPERTIES ANALYSIS (≥ $650K)")
print("="*60)

if price_col:
    df['high_value'] = df[price_col] >= 650000
    high_value = df[df['high_value']]
    regular_value = df[~df['high_value']]
    
    print(f"High-value properties (≥ $650K): {len(high_value):,}")
    print(f"Regular properties (< $650K): {len(regular_value):,}")
    print(f"Percentage high-value: {len(high_value)/len(df)*100:.1f}%")
    
    print("\nComparison - High Value vs Regular Properties:")
    
    # Define comparison columns based on available data
    possible_comp_cols = ['grade', 'sqft_living', 'bathrooms', 'bedrooms', 
                         'waterfront', 'view', 'condition', 'sqft_above',
                         'sqft_lot', 'floors', 'yr_built']
    
    comp_cols = [col for col in possible_comp_cols if col in df.columns]
    
    if comp_cols:
        comparison_data = []
        for col in comp_cols:
            hv_mean = high_value[col].mean() if len(high_value) > 0 else 0
            rv_mean = regular_value[col].mean() if len(regular_value) > 0 else 0
            
            if rv_mean != 0:  # Avoid division by zero
                pct_diff = ((hv_mean - rv_mean) / rv_mean) * 100
            else:
                pct_diff = 0
                
            comparison_data.append({
                'Feature': col,
                'High_Value_Mean': hv_mean,
                'Regular_Mean': rv_mean,
                'Difference': hv_mean - rv_mean,
                'Pct_Difference (%)': pct_diff
            })
        
        comparison_df = pd.DataFrame(comparison_data)
        print(comparison_df.to_string(index=False))
        
        # Visualize top differences
        top_diff = comparison_df.nlargest(5, 'Pct_Difference (%)')
        if len(top_diff) > 0:
            fig, ax = plt.subplots(figsize=(10, 6))
            y_pos = np.arange(len(top_diff))
            ax.barh(y_pos, top_diff['Pct_Difference (%)'])
            ax.set_yticks(y_pos)
            ax.set_yticklabels(top_diff['Feature'])
            ax.set_xlabel('Percentage Difference (%)')
            ax.set_title('Top Features Differentiating High-Value Properties')
            ax.invert_yaxis()  # Highest on top
            plt.tight_layout()
            plt.show()
    else:
        print("No comparison columns found in dataset")
else:
    print(f"Cannot analyze high-value properties: Price column not found")

# =========== PART 2: MODELING (Based on EDA insights) ===========
print("\n" + "="*60)
print("MODELING PHASE")
print("="*60)

if price_col and len(numerical_cols) > 2:
    from sklearn.model_selection import train_test_split, cross_val_score
    from sklearn.preprocessing import StandardScaler
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.linear_model import LinearRegression, Ridge
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.neighbors import KNeighborsRegressor
    from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
    
    # Select features based on correlation analysis
    if 'price_corr' in locals() and len(price_corr) > 1:
        # Use top 10 features excluding price itself
        top_features = price_corr.index[1:11].tolist() if len(price_corr) > 10 else price_corr.index[1:].tolist()
    else:
        # Fallback: use all numerical columns except price and ID
        top_features = [col for col in numerical_cols if col != price_col and 'id' not in col.lower()]
        top_features = top_features[:10]  # Take first 10
    
    print(f"\nSelected features for modeling ({len(top_features)}):")
    print(top_features)
    
    X = df[top_features]
    y = df[price_col]
    
    # Handle any missing values
    if X.isnull().sum().sum() > 0:
        print(f"\nFilling {X.isnull().sum().sum()} missing values with median...")
        X = X.fillna(X.median())
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    print(f"\nTrain set: {X_train.shape[0]} samples")
    print(f"Test set: {X_test.shape[0]} samples")
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train multiple models
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(alpha=1.0),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
        'Decision Tree': DecisionTreeRegressor(random_state=42),
        'K-Nearest Neighbors': KNeighborsRegressor(n_neighbors=5)
    }
    
    print("\n" + "="*60)
    print("MODEL PERFORMANCE COMPARISON")
    print("="*60)
    
    results = []
    for name, model in models.items():
        print(f"\nTraining {name}...")
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Cross-validation
        cv_scores = cross_val_score(model, X_train_scaled, y_train, 
                                    cv=5, scoring='r2', n_jobs=-1)
        
        results.append({
            'Model': name,
            'RMSE': f"${rmse:,.0f}",
            'MAE': f"${mae:,.0f}",
            'R²': f"{r2:.3f}",
            'CV R² Mean': f"{cv_scores.mean():.3f}",
            'CV R² Std': f"{cv_scores.std():.3f}"
        })
        
        print(f"  R²: {r2:.3f}")
        print(f"  RMSE: ${rmse:,.0f}")
        print(f"  MAE: ${mae:,.0f}")
        print(f"  CV R²: {cv_scores.mean():.3f} (±{cv_scores.std():.3f})")
    
    # Display results
    results_df = pd.DataFrame(results)
    print("\n" + "="*60)
    print("SUMMARY TABLE")
    print("="*60)
    print(results_df.to_string(index=False))
    
    # Feature importance from Random Forest
    print("\n" + "="*60)
    print("FEATURE IMPORTANCE (Random Forest)")
    print("="*60)
    
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train_scaled, y_train)
    
    feature_importance = pd.DataFrame({
        'Feature': top_features,
        'Importance': rf_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print(feature_importance.to_string(index=False))
    
    # Visualize feature importance
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance['Feature'], feature_importance['Importance'])
    plt.xlabel('Importance')
    plt.title('Feature Importance - Random Forest')
    plt.gca().invert_yaxis()  # Highest importance on top
    plt.tight_layout()
    plt.show()
    
    # =========== PART 3: BUSINESS INSIGHTS ===========
    print("\n" + "="*60)
    print("KEY BUSINESS INSIGHTS FOR MANAGEMENT")
    print("="*60)
    
    if 'high_value' in df.columns:
        print("\n1. HIGH-VALUE PROPERTIES (≥ $650K):")
        print(f"   • Represent {len(high_value)/len(df)*100:.1f}% of all properties")
        print(f"   • Average price: ${high_value[price_col].mean():,.0f}")
        print(f"   • Key differentiators:")
        
        # Show top 3 differentiating features
        if 'comparison_df' in locals():
            top_diff_features = comparison_df.nlargest(3, 'Pct_Difference (%)')
            for _, row in top_diff_features.iterrows():
                print(f"     - {row['Feature']}: {row['High_Value_Mean']:.1f} vs {row['Regular_Mean']:.1f} ({row['Pct_Difference (%)']:+.1f}%)")
    
    print("\n2. PREDICTIVE MODELING RESULTS:")
    best_model_row = results_df.loc[results_df['R²'].astype(float).idxmax()]
    print(f"   • Best performing model: {best_model_row['Model']}")
    print(f"   • R² Score: {best_model_row['R²']}")
    print(f"   • Average prediction error: {best_model_row['MAE']}")
    
    print("\n3. TOP PRICE PREDICTORS:")
    for i, row in feature_importance.head(3).iterrows():
        print(f"   • {row['Feature']} (importance: {row['Importance']:.3f})")
    
    print("\n4. RECOMMENDATIONS:")
    print("   • Focus on improving top predictors (see above) for property value appreciation")
    print("   • High-value properties show distinct characteristics - target these features")
    print("   • Use the Random Forest model for accurate price predictions")
    print("   • Monitor waterfront and grade as key value drivers")
    
    # Prepare for Tableau (optional)
    print("\n" + "="*60)
    print("TABLEAU PREPARATION")
    print("="*60)
    
    tableau_df = df.copy()
    
    # Create useful derived columns for Tableau
    if price_col:
        tableau_df['price_category'] = np.where(tableau_df[price_col] >= 650000, 
                                               'High (≥$650K)', 'Regular (<$650K)')
        
        # Create price per sqft if sqft_living exists
        if 'sqft_living' in tableau_df.columns:
            tableau_df['price_per_sqft'] = tableau_df[price_col] / tableau_df['sqft_living']
            print("   • Created 'price_per_sqft' column")
        
        print("   • Created 'price_category' column")
    
    # Save for Tableau
    tableau_df.to_csv('../data/house_data_tableau_ready.csv', index=False)
    print(f"   • Tableau-ready data saved to: ../data/house_data_tableau_ready.csv")
    
else:
    print("Skipping modeling phase - insufficient data or price column not found")
    print(f"Price column found: {price_col}")
    print(f"Numerical columns: {len(numerical_cols)}")

print("\n" + "="*60)
print("ANALYSIS COMPLETE!")
print("="*60)

Dataset: 21596 rows, 21 columns

Columns in dataset:
['7129300520', '10/13/14', '3', '1', '1180', '5650', '1.1', '0', '0.1', '3.1', '7', '1180.1', '0.2', '1955', '0.3', '98178', '47.5112', '-122.257', '1340', '5650.1', '221900']

MISSING VALUES ANALYSIS
Empty DataFrame
Columns: [Missing Count, Missing %]
Index: []
✓ No missing values found!

BASIC STATISTICAL SUMMARY
Numerical columns (20): ['7129300520', '3', '1', '1180', '5650', '1.1', '0', '0.1', '3.1', '7', '1180.1', '0.2', '1955', '0.3', '98178', '47.5112', '-122.257', '1340', '5650.1', '221900']
Categorical columns (1): ['10/13/14']

Basic statistics for key variables:
         7129300520         3         1      1180        5650       1.1  \
count  2.159600e+04  21596.00  21596.00  21596.00    21596.00  21596.00   
mean   4.580356e+09      3.37      2.12   2080.36    15099.85      1.49   
std    2.876750e+09      0.93      0.77    918.11    41413.55      0.54   
min    1.000102e+06      1.00      0.50    370.00      520.00      

In [2]:
# === STEP 1: EXPLORE YOUR DATASET STRUCTURE ===
import pandas as pd

# Load the data
df = pd.read_csv('data/regression_data.csv')

print("="*60)
print("DATASET EXPLORATION")
print("="*60)

print(f"Dataset shape: {df.shape}")
print(f"Columns ({len(df.columns)}):")

# Display all columns with their data types
for i, col in enumerate(df.columns, 1):
    print(f"{i:3}. {col:20} - {df[col].dtype}")

print("\n" + "="*60)
print("LOOKING FOR PRICE-RELATED COLUMNS")
print("="*60)

# Look for columns that might contain price information
price_keywords = ['price', 'Price', 'PRICE', 'value', 'Value', 'VALUE', 
                  'selling', 'Selling', 'sale', 'Sale', 'cost', 'Cost']

price_columns = []
for col in df.columns:
    col_lower = col.lower()
    for keyword in price_keywords:
        if keyword.lower() in col_lower:
            price_columns.append(col)
            break

if price_columns:
    print(f"Found potential price columns: {price_columns}")
    for col in price_columns:
        print(f"\nStatistics for '{col}':")
        print(df[col].describe())
else:
    print("No obvious price columns found. Showing first 5 rows:")
    print(df.head())

print("\n" + "="*60)
print("COLUMNS AVAILABLE")
print("="*60)
print("All columns in your dataset:")
for col in df.columns:
    print(f"- '{col}'")

DATASET EXPLORATION
Dataset shape: (21596, 21)
Columns (21):
  1. 7129300520           - int64
  2. 10/13/14             - object
  3. 3                    - int64
  4. 1                    - float64
  5. 1180                 - int64
  6. 5650                 - int64
  7. 1.1                  - float64
  8. 0                    - int64
  9. 0.1                  - int64
 10. 3.1                  - int64
 11. 7                    - int64
 12. 1180.1               - int64
 13. 0.2                  - int64
 14. 1955                 - int64
 15. 0.3                  - int64
 16. 98178                - int64
 17. 47.5112              - float64
 18. -122.257             - float64
 19. 1340                 - int64
 20. 5650.1               - int64
 21. 221900               - int64

LOOKING FOR PRICE-RELATED COLUMNS
No obvious price columns found. Showing first 5 rows:
   7129300520 10/13/14  3     1  1180    5650  1.1  0  0.1  3.1  ...  1180.1  \
0  6414100192  12/9/14  3  2.25  2570    7242  

In [3]:
# === FLEXIBLE TABLEAU DATA PREPARATION ===
import pandas as pd
import numpy as np

# Load your data
df = pd.read_csv('data/regression_data.csv')

# Identify the target price column
def find_price_column(df):
    """Find the price/target column in the dataset"""
    price_keywords = ['price', 'Price', 'PRICE', 'value', 'Value', 'target', 
                     'Target', 'y', 'Y', 'selling', 'sale', 'cost']
    
    # First, check for exact matches
    for col in df.columns:
        if col.lower() in ['price', 'target', 'y']:
            return col
    
    # Check for partial matches
    for col in df.columns:
        col_lower = col.lower()
        for keyword in price_keywords:
            if keyword in col_lower:
                return col
    
    # If no price column found, use the last numerical column
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    if len(numerical_cols) > 0:
        print(f"⚠ Warning: Using '{numerical_cols[-1]}' as target variable")
        return numerical_cols[-1]
    
    return None

# Find the price column
price_col = find_price_column(df)
print(f"Target column identified: '{price_col}'")

# Let's see what other key columns we have
print("\n" + "="*60)
print("IDENTIFYING KEY FEATURES")
print("="*60)

# Common feature names in house price datasets
feature_mapping = {
    'sqft_living': ['sqft_living', 'living_area', 'area_living', 'sqftliving'],
    'sqft_lot': ['sqft_lot', 'lot_area', 'area_lot', 'sqftlot'],
    'bedrooms': ['bedrooms', 'bedroom', 'beds', 'bed'],
    'bathrooms': ['bathrooms', 'bathroom', 'baths', 'bath'],
    'grade': ['grade', 'Grade', 'GRADE', 'overall_grade'],
    'waterfront': ['waterfront', 'Waterfront', 'WATERFRONT', 'view_water'],
    'condition': ['condition', 'Condition', 'CONDITION', 'overall_cond'],
    'view': ['view', 'View', 'VIEW'],
    'floors': ['floors', 'Floors', 'FLOORS', 'stories'],
    'yr_built': ['yr_built', 'year_built', 'built_year', 'yearbuilt'],
    'sqft_above': ['sqft_above', 'above_area', 'area_above'],
    'sqft_basement': ['sqft_basement', 'basement_area', 'area_basement'],
    'sqft_living15': ['sqft_living15', 'living_area_2015'],
    'sqft_lot15': ['sqft_lot15', 'lot_area_2015']
}

# Map actual column names to standard names
standard_cols = {}
for standard_name, possible_names in feature_mapping.items():
    found = False
    for possible in possible_names:
        if possible in df.columns:
            standard_cols[standard_name] = possible
            found = True
            print(f"✓ {standard_name}: '{possible}'")
            break
    if not found:
        print(f"✗ {standard_name}: Not found")

# Create Tableau-optimized dataset
tableau_df = df.copy()

# Rename columns to standard names for easier reference
for std_name, actual_name in standard_cols.items():
    if std_name != actual_name:
        tableau_df[std_name] = tableau_df[actual_name]

# 1. Create price categories - USING THE IDENTIFIED PRICE COLUMN
print(f"\nUsing '{price_col}' for price analysis...")

# Check price distribution to set appropriate bins
if price_col:
    price_stats = tableau_df[price_col].describe()
    min_price = price_stats['min']
    max_price = price_stats['max']
    median_price = price_stats['50%']
    
    print(f"Price statistics:")
    print(f"  Min: ${min_price:,.0f}")
    print(f"  Max: ${max_price:,.0f}")
    print(f"  Median: ${median_price:,.0f}")
    
    # Create dynamic bins based on data
    if max_price <= 1000000:
        bins = [0, 300000, 500000, 650000, max_price]
        labels = ['< $300K', '$300K-500K', '$500K-650K', f'$650K-${max_price/1000:.0f}K']
    else:
        bins = [0, 300000, 500000, 650000, 1000000, max_price]
        labels = ['< $300K', '$300K-500K', '$500K-650K', '$650K-1M', f'> $1M']
    
    tableau_df['price_category'] = pd.cut(tableau_df[price_col], bins=bins, labels=labels)
    print(f"Created price categories: {labels}")

# 2. Create size categories if sqft_living exists
if 'sqft_living' in tableau_df.columns:
    # Get living area statistics
    living_stats = tableau_df['sqft_living'].describe()
    
    # Create dynamic bins
    max_living = living_stats['max']
    if max_living <= 5000:
        bins = [0, 1000, 2000, 3000, max_living]
        labels = ['<1K sqft', '1K-2K sqft', '2K-3K sqft', f'3K-{max_living/1000:.0f}K sqft']
    else:
        bins = [0, 1000, 2000, 3000, 4000, max_living]
        labels = ['<1K sqft', '1K-2K sqft', '2K-3K sqft', '3K-4K sqft', f'>4K sqft']
    
    tableau_df['size_category'] = pd.cut(tableau_df['sqft_living'], bins=bins, labels=labels)
    print(f"Created size categories: {labels}")

# 3. Create house age if yr_built exists
if 'yr_built' in tableau_df.columns:
    tableau_df['house_age'] = 2015 - tableau_df['yr_built']  # Using 2015 as reference
    
    # Create age categories
    age_bins = [0, 5, 10, 20, 30, 50, 100, float('inf')]
    age_labels = ['<5 yrs', '5-10 yrs', '10-20 yrs', '20-30 yrs', '30-50 yrs', '50-100 yrs', '>100 yrs']
    tableau_df['age_category'] = pd.cut(tableau_df['house_age'], bins=age_bins, labels=age_labels)
    print(f"Created age categories")

# 4. Calculate price per square foot if both price and sqft_living exist
if price_col and 'sqft_living' in tableau_df.columns:
    tableau_df['price_per_sqft'] = tableau_df[price_col] / tableau_df['sqft_living']
    print(f"Created 'price_per_sqft' column")

# 5. Create boolean flags
if 'waterfront' in tableau_df.columns:
    tableau_df['is_waterfront'] = tableau_df['waterfront'] == 1
    print(f"Created 'is_waterfront' flag")

if 'yr_renovated' in tableau_df.columns:
    tableau_df['is_renovated'] = tableau_df['yr_renovated'] > 0
    print(f"Created 'is_renovated' flag")

if 'grade' in tableau_df.columns:
    tableau_df['high_grade'] = tableau_df['grade'] >= 10
    print(f"Created 'high_grade' flag")

if price_col:
    tableau_df['is_high_value'] = tableau_df[price_col] >= 650000
    print(f"Created 'is_high_value' flag (≥ $650K)")

# 6. Extract date components if date column exists
date_columns = [col for col in tableau_df.columns if 'date' in col.lower()]
if date_columns:
    date_col = date_columns[0]
    try:
        tableau_df[date_col] = pd.to_datetime(tableau_df[date_col])
        tableau_df['year'] = tableau_df[date_col].dt.year
        tableau_df['month'] = tableau_df[date_col].dt.month
        tableau_df['month_name'] = tableau_df[date_col].dt.month_name()
        
        # Create season
        def get_season(month):
            if month in [12, 1, 2]:
                return 'Winter'
            elif month in [3, 4, 5]:
                return 'Spring'
            elif month in [6, 7, 8]:
                return 'Summer'
            else:
                return 'Fall'
        
        tableau_df['season'] = tableau_df['month'].apply(get_season)
        print(f"Created date-based columns from '{date_col}'")
    except:
        print(f"Could not parse date column '{date_col}'")

# 7. Create view quality categories
if 'view' in tableau_df.columns:
    # Check view value range
    view_values = tableau_df['view'].unique()
    if len(view_values) <= 6:
        # Create categorical mapping
        view_labels = {0: 'No View', 1: 'Fair View', 2: 'Average View', 
                      3: 'Good View', 4: 'Excellent View'}
        tableau_df['view_category'] = tableau_df['view'].map(view_labels)
        print(f"Created 'view_category'")

# 8. Create condition categories
if 'condition' in tableau_df.columns:
    condition_values = tableau_df['condition'].unique()
    if len(condition_values) <= 5:
        condition_labels = {1: 'Poor', 2: 'Below Avg', 3: 'Average', 
                          4: 'Good', 5: 'Excellent'}
        tableau_df['condition_category'] = tableau_df['condition'].map(condition_labels)
        print(f"Created 'condition_category'")

# 9. Calculate bedroom-to-bathroom ratio if both exist
if 'bedrooms' in tableau_df.columns and 'bathrooms' in tableau_df.columns:
    tableau_df['bed_bath_ratio'] = tableau_df['bedrooms'] / tableau_df['bathrooms'].replace(0, np.nan)
    print(f"Created 'bed_bath_ratio'")

# 10. Create location segments if coordinates exist
if 'lat' in tableau_df.columns and 'long' in tableau_df.columns:
    if tableau_df['lat'].notna().any() and tableau_df['long'].notna().any():
        # Create simple location categories
        lat_median = tableau_df['lat'].median()
        long_median = tableau_df['long'].median()
        
        tableau_df['location_quadrant'] = np.where(
            (tableau_df['lat'] >= lat_median) & (tableau_df['long'] >= long_median),
            'NE',
            np.where(
                (tableau_df['lat'] >= lat_median) & (tableau_df['long'] < long_median),
                'NW',
                np.where(
                    (tableau_df['lat'] < lat_median) & (tableau_df['long'] >= long_median),
                    'SE',
                    'SW'
                )
            )
        )
        print(f"Created 'location_quadrant'")

# Save for Tableau
output_file = 'data/house_data_tableau_ready.csv'
tableau_df.to_csv(output_file, index=False)

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"✓ Tableau-ready dataset saved to: {output_file}")
print(f"  Original shape: {df.shape}")
print(f"  New shape: {tableau_df.shape}")
print(f"  Price column used: '{price_col}'")

# Show new columns created
new_columns = [col for col in tableau_df.columns if col not in df.columns]
print(f"\nNew columns created ({len(new_columns)}):")
for col in new_columns:
    print(f"  - {col}")

# Show sample of prepared data
print(f"\nSample of prepared data (first 3 rows):")
print(tableau_df[new_columns[:5]].head(3) if new_columns else tableau_df.head(3))

print("\n" + "="*60)
print("NEXT STEPS FOR TABLEAU")
print("="*60)
print("1. Open Tableau Desktop")
print("2. Connect to: data/house_data_tableau_ready.csv")
print(f"3. Use '{price_col}' as your target variable")
print("4. Use the created categories (price_category, size_category, etc.) for filtering")
print("5. Create dashboards as per the previous instructions")

⚠ Warning: Using '221900' as target variable
Target column identified: '221900'

IDENTIFYING KEY FEATURES
✗ sqft_living: Not found
✗ sqft_lot: Not found
✗ bedrooms: Not found
✗ bathrooms: Not found
✗ grade: Not found
✗ waterfront: Not found
✗ condition: Not found
✗ view: Not found
✗ floors: Not found
✗ yr_built: Not found
✗ sqft_above: Not found
✗ sqft_basement: Not found
✗ sqft_living15: Not found
✗ sqft_lot15: Not found

Using '221900' for price analysis...
Price statistics:
  Min: $78,000
  Max: $7,700,000
  Median: $450,000
Created price categories: ['< $300K', '$300K-500K', '$500K-650K', '$650K-1M', '> $1M']
Created 'is_high_value' flag (≥ $650K)

SUMMARY
✓ Tableau-ready dataset saved to: data/house_data_tableau_ready.csv
  Original shape: (21596, 21)
  New shape: (21596, 23)
  Price column used: '221900'

New columns created (2):
  - price_category
  - is_high_value

Sample of prepared data (first 3 rows):
  price_category  is_high_value
0     $500K-650K          False
1        <

In [4]:
# === CORRECTED TABLEAU DATA PREPARATION ===
import pandas as pd
import numpy as np

# Load data with proper header handling
try:
    # Try reading with header
    df = pd.read_csv('data/regression_data.csv')
    
    # Check if first row looks like data (contains numbers)
    first_row_numeric = pd.to_numeric(df.iloc[0], errors='coerce').notna().any()
    
    if first_row_numeric:
        print("First row appears to contain data, not column names")
        print("Trying to load with proper column names...")
        
        # Read without header and assign proper column names
        df = pd.read_csv('data/regression_data.csv', header=None)
        
        # Based on common house price dataset columns
        common_columns = [
            'id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 
            'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
            'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
            'lat', 'long', 'sqft_living15', 'sqft_lot15'
        ]
        
        # Use appropriate number of columns
        if len(df.columns) <= len(common_columns):
            df.columns = common_columns[:len(df.columns)]
        else:
            # Use generic column names
            df.columns = [f'col_{i}' for i in range(len(df.columns))]
            
        print(f"Assigned column names: {list(df.columns)}")
        
except Exception as e:
    print(f"Error loading file: {e}")
    # Try alternative approach
    try:
        df = pd.read_csv('data/regression_data.csv', engine='python')
    except:
        df = pd.read_excel('data/regression_data.xls')

print("\n" + "="*60)
print("DATASET STRUCTURE")
print("="*60)
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nFirst 3 rows:")
print(df.head(3))
print(f"\nData types:")
print(df.dtypes)

# Let's find the actual price column
print("\n" + "="*60)
print("FINDING PRICE COLUMN")
print("="*60)

# Look for columns that could be price
potential_price_cols = []
for col in df.columns:
    try:
        # Convert to numeric to check
        numeric_col = pd.to_numeric(df[col], errors='coerce')
        if numeric_col.notna().any():
            # Check if values look like prices (typical range)
            median_val = numeric_col.median()
            if 50000 < median_val < 5000000:  # Reasonable price range
                potential_price_cols.append((col, median_val, numeric_col.dtype))
    except:
        continue

if potential_price_cols:
    print("Potential price columns found:")
    for col, median_val, dtype in potential_price_cols:
        print(f"  '{col}': median = ${median_val:,.0f}, type = {dtype}")
    
    # Select the most likely price column
    # Prefer columns named 'price' or with highest median in reasonable range
    price_col = None
    for col, median_val, dtype in potential_price_cols:
        if 'price' in str(col).lower():
            price_col = col
            break
    
    if not price_col:
        # Choose column with median closest to typical house price
        price_col = max(potential_price_cols, key=lambda x: x[1] if 100000 < x[1] < 2000000 else 0)[0]
    
    print(f"\nSelected price column: '{price_col}'")
else:
    print("No obvious price column found. Using last numeric column.")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    price_col = numeric_cols[-1] if len(numeric_cols) > 0 else df.columns[-1]
    print(f"Using '{price_col}' as price column")

# Convert price column to numeric
df[price_col] = pd.to_numeric(df[price_col], errors='coerce')

print("\n" + "="*60)
print("PRICE STATISTICS")
print("="*60)
print(f"Column: {price_col}")
print(f"Min: ${df[price_col].min():,.0f}")
print(f"Max: ${df[price_col].max():,.0f}")
print(f"Mean: ${df[price_col].mean():,.0f}")
print(f"Median: ${df[price_col].median():,.0f}")
print(f"Missing values: {df[price_col].isna().sum()}")

# === CREATE TABLEAU-OPTIMIZED DATASET ===
tableau_df = df.copy()

# Standardize column names (remove any leading/trailing spaces)
tableau_df.columns = [str(col).strip() for col in tableau_df.columns]
price_col = price_col.strip()

# 1. Create price categories
price_data = tableau_df[price_col].dropna()
if len(price_data) > 0:
    min_price = price_data.min()
    max_price = price_data.max()
    
    # Create dynamic bins
    if max_price <= 1000000:
        bins = [min_price-1, 300000, 500000, 650000, max_price]
        labels = ['< $300K', '$300K-500K', '$500K-650K', f'≥ $650K']
    else:
        bins = [min_price-1, 300000, 500000, 650000, 1000000, max_price]
        labels = ['< $300K', '$300K-500K', '$500K-650K', '$650K-1M', '> $1M']
    
    tableau_df['price_category'] = pd.cut(tableau_df[price_col], bins=bins, labels=labels, include_lowest=True)
    print(f"\nCreated price categories: {labels}")

# 2. Identify and create other useful categories
print("\n" + "="*60)
print("CREATING ADDITIONAL CATEGORIES")
print("="*60)

# Find living area column
living_area_cols = [col for col in tableau_df.columns if any(term in str(col).lower() 
                    for term in ['sqft', 'area', 'living', 'size'])]
if living_area_cols:
    living_col = living_area_cols[0]
    print(f"Using '{living_col}' for living area")
    
    # Create size categories
    if pd.api.types.is_numeric_dtype(tableau_df[living_col]):
        living_data = tableau_df[living_col].dropna()
        if len(living_data) > 0:
            bins = [0, 1000, 2000, 3000, 4000, float('inf')]
            labels = ['<1K sqft', '1K-2K sqft', '2K-3K sqft', '3K-4K sqft', '>4K sqft']
            tableau_df['size_category'] = pd.cut(tableau_df[living_col], bins=bins, labels=labels)
            print(f"Created size categories")
            
            # Calculate price per sqft
            tableau_df['price_per_sqft'] = tableau_df[price_col] / tableau_df[living_col]
            print(f"Created price_per_sqft")

# Find bedroom and bathroom columns
bedroom_cols = [col for col in tableau_df.columns if any(term in str(col).lower() 
                for term in ['bed', 'room', 'brm'])]
bathroom_cols = [col for col in tableau_df.columns if any(term in str(col).lower() 
                 for term in ['bath', 'bth'])]

if bedroom_cols and bathroom_cols:
    bedroom_col = bedroom_cols[0]
    bathroom_col = bathroom_cols[0]
    tableau_df['bed_bath_ratio'] = pd.to_numeric(tableau_df[bedroom_col], errors='coerce') / \
                                   pd.to_numeric(tableau_df[bathroom_col].replace(0, np.nan), errors='coerce')
    print(f"Created bed_bath_ratio")

# Find grade/condition columns
for grade_term in ['grade', 'rating', 'score']:
    grade_cols = [col for col in tableau_df.columns if grade_term in str(col).lower()]
    if grade_cols:
        grade_col = grade_cols[0]
        if pd.api.types.is_numeric_dtype(tableau_df[grade_col]):
            tableau_df['high_grade'] = tableau_df[grade_col] >= tableau_df[grade_col].median()
            print(f"Created high_grade based on '{grade_col}'")
        break

# Find waterfront/view columns
for waterfront_term in ['waterfront', 'water', 'view']:
    waterfront_cols = [col for col in tableau_df.columns if waterfront_term in str(col).lower()]
    if waterfront_cols:
        waterfront_col = waterfront_cols[0]
        if pd.api.types.is_numeric_dtype(tableau_df[waterfront_col]):
            tableau_df['has_water_view'] = tableau_df[waterfront_col] > 0
            print(f"Created has_water_view based on '{waterfront_col}'")
        break

# High value flag
tableau_df['is_high_value'] = tableau_df[price_col] >= 650000
print(f"Created is_high_value flag")

# Find date column
date_cols = [col for col in tableau_df.columns if any(term in str(col).lower() 
             for term in ['date', 'time', 'year', 'month'])]
if date_cols:
    date_col = date_cols[0]
    try:
        tableau_df[date_col] = pd.to_datetime(tableau_df[date_col], errors='coerce')
        tableau_df['sale_year'] = tableau_df[date_col].dt.year
        tableau_df['sale_month'] = tableau_df[date_col].dt.month
        tableau_df['sale_quarter'] = tableau_df[date_col].dt.quarter
        print(f"Created date-based columns from '{date_col}'")
    except:
        print(f"Could not parse date column '{date_col}'")

# === SAVE FOR TABLEAU ===
output_file = 'data/house_data_tableau_fixed.csv'
tableau_df.to_csv(output_file, index=False)

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"✓ Tableau-ready dataset saved to: {output_file}")
print(f"  Original shape: {df.shape}")
print(f"  New shape: {tableau_df.shape}")
print(f"  Price column: '{price_col}'")

# Show all columns
print(f"\nAll columns in Tableau-ready dataset ({len(tableau_df.columns)}):")
for i, col in enumerate(tableau_df.columns, 1):
    print(f"{i:3}. {col}")

# Show sample
print(f"\nSample data (first 2 rows):")
pd.set_option('display.max_columns', None)
print(tableau_df.head(2))
pd.reset_option('display.max_columns')

print("\n" + "="*60)
print("TABLEAU PREPARATION GUIDE")
print("="*60)
print("1. Open Tableau Desktop")
print(f"2. Connect to: {output_file}")
print(f"3. Main variables to use:")
print(f"   - Target variable: '{price_col}'")
print(f"   - Price categories: 'price_category'")
print(f"   - High-value filter: 'is_high_value'")
if 'size_category' in tableau_df.columns:
    print(f"   - Size analysis: 'size_category'")
if 'price_per_sqft' in tableau_df.columns:
    print(f"   - Efficiency metric: 'price_per_sqft'")

print("\n4. For high-value analysis (≥ $650K):")
print("   - Use 'is_high_value' as filter")
print("   - Compare with 'price_category'")
print("   - Analyze correlations with other features")

print("\n5. Dashboard creation steps:")
print("   a. Create KPI cards for counts and averages")
print("   b. Create distribution charts for price_category")
print("   c. Create comparison charts (high-value vs regular)")
print("   d. Add filters for interactive exploration")
print("   e. Create geographic map if lat/long columns exist")

First row appears to contain data, not column names
Trying to load with proper column names...
Assigned column names: ['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']

DATASET STRUCTURE
Shape: (21597, 21)
Columns: ['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']

First 3 rows:
           id      date  price  bedrooms  bathrooms  sqft_living  sqft_lot  \
0  7129300520  10/13/14      3      1.00       1180         5650       1.0   
1  6414100192   12/9/14      3      2.25       2570         7242       2.0   
2  5631500400   2/25/15      2      1.00        770        10000       1.0   

   floors  wate

C:\Users\profe\AppData\Local\Temp\ipykernel_34236\1337167597.py:203: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  tableau_df[date_col] = pd.to_datetime(tableau_df[date_col], errors='coerce')



SUMMARY
✓ Tableau-ready dataset saved to: data/house_data_tableau_fixed.csv
  Original shape: (21597, 21)
  New shape: (21597, 31)
  Price column: 'sqft_lot15'

All columns in Tableau-ready dataset (31):
  1. id
  2. date
  3. price
  4. bedrooms
  5. bathrooms
  6. sqft_living
  7. sqft_lot
  8. floors
  9. waterfront
 10. view
 11. condition
 12. grade
 13. sqft_above
 14. sqft_basement
 15. yr_built
 16. yr_renovated
 17. zipcode
 18. lat
 19. long
 20. sqft_living15
 21. sqft_lot15
 22. price_category
 23. size_category
 24. price_per_sqft
 25. bed_bath_ratio
 26. high_grade
 27. has_water_view
 28. is_high_value
 29. sale_year
 30. sale_month
 31. sale_quarter

Sample data (first 2 rows):
           id       date  price  bedrooms  bathrooms  sqft_living  sqft_lot  \
0  7129300520 2014-10-13      3      1.00       1180         5650       1.0   
1  6414100192 2014-12-09      3      2.25       2570         7242       2.0   

   floors  waterfront  view  condition  grade  sqft_above 